# Bayes Theorem for Predicting the Probability of an Email Being Spam

S = Spam
w = Word

$P(Spam|w_{1}, w_{2},..., w_{n}) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_{i}|Spam)$

The probability that an email consisting of the words $w_{1}, w_{2},... w_{n}$ is proportional to the probability that any given email is spam multiplied by the product of each word's probability to appear in a spam email.



In [1]:
import pandas as pd
import math

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

pd.options.mode.chained_assignment = None  # default='warn'

PREDICTION = 'Prediction'
CLASSIFICATION = 'Classiciation'

## Functions

In [2]:
def count_vocab(emails):
    total_words = 0
    
    for index, row in emails.iterrows():
        total_words += sum(row.values[1:-2])
            
    return total_words

In [3]:
def calculate_word_spamicity(w_spam_count, vocab, spam_vocab):
    alpha = 1
    
    spamicity = (w_spam_count + alpha) / (spam_vocab + alpha * vocab)
    return spamicity

In [4]:
def build_word_spamicity_dict(spam_emails, vocab, spam_vocab):
    spam_word_appearances = {}
    
    for (column_name, column_data) in spam_emails.iteritems():
        if column_name != 'Email No.' and column_name != PREDICTION and column_name != CLASSIFICATION:
            spam_word_appearances[column_name] = sum(column_data.values)
            
    for word in spam_word_appearances:
        spam_word_appearances[word] = calculate_word_spamicity(spam_word_appearances[word], vocab, spam_vocab)
            
    return spam_word_appearances

In [5]:
def calculate_emails(testing_emails, word_spamicities, spam_proportion):
    test_data_map = {}
    for index, email in testing_emails.iterrows():
        email_spamicity = math.log(spam_proportion)
        for column in testing_data.columns[1:-2]:
            if email[column] > 0:
                # log P(spam) + sum log P(w|spam)
                email_spamicity += math.log(word_spamicities[column])*email[column]

        test_data_map[index] = email_spamicity
        
    return test_data_map

In [6]:
def calculate_accuracy(testing_emails):
    number_correct = 0
    for index, email in testing_emails.iterrows():
        if email[PREDICTION] == email[CLASSIFICATION]:
            number_correct += 1
        
    return number_correct / testing_data.shape[0] * 100

# Model

## Step 1: Partition the data into training and test segments

20% of the data for testing, and the remaining 80% is training (i.e. the 80% training data will confirm whether the 20% testing data labels are correct).

In [7]:
df = pd.read_csv('emails.csv')
total_num_emails = df.shape[0]

end = total_num_emails//5

# Take out all stopwords
for col in df.columns:
    if col in stop_words:
        df.drop(col, axis=1, inplace=True)

testing_data = df.iloc[:end].copy()

# This is where the model's prediction will be stored
testing_data[CLASSIFICATION] = ""

training_data = df.iloc[end:]

testing_data


,Email No.,ect,hou,enron,com,gas,deal,meter,hpl,please,...,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction,Classiciation
0,Email 1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
1,Email 2,24,27,1,3,1,0,0,0,2,...,0,0,0,0,0,0,1,0,0,
2,Email 3,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,
3,Email 4,22,10,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,
4,Email 5,17,9,0,0,2,0,3,0,1,...,0,0,0,0,0,0,1,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Email 1030,4,0,0,2,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,
1030,Email 1031,18,11,59,7,0,0,0,0,4,...,0,0,2,0,0,0,2,0,0,
1031,Email 1032,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,
1032,Email 1033,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,2,0,1,


In [8]:
training_data

,Email No.,ect,hou,enron,com,gas,deal,meter,hpl,please,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
1034,Email 1035,18,0,2,7,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
1035,Email 1036,3,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1036,Email 1037,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1037,Email 1038,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1038,Email 1039,11,4,0,0,4,6,11,3,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,11,3,1,3,5,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,1,0,0,4,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,1


## Step 2: Get probabilities that any one email in the training data is either spam or ham

In the labelled dataset, count the number of spam and ham emails.

$P(Spam) = \frac{Spam\,Emails}{Total\,Emails}$

In [9]:
spam_proportion = training_data['Prediction'].value_counts()[1] / training_data.shape[0]
spam_proportion

0.291445142580957

$P(Ham) = \frac{Ham\,Emails}{Total\,Emails}$

In [10]:
ham_proportion = training_data['Prediction'].value_counts()[0] / training_data.shape[0]
ham_proportion

0.708554857419043

## Step 3: Get the "spamicity" and "hamicity" probability of each word in the testing data email

**w** = word
<br>**vocab** = total words in dataset
<br>**spam_vocab**
<br>**wi_spam_count**

Count all unique words in the labelled dataset to get **vocab**.

Count the total number of words in labelled spam emails (ignoring uniqueness) to get **spam_vocab**.

For each word **w**, count all instances of the word in the spam emails to get **wi_spam_count**.

Calculate spamicity of each word and store the word and its spamicity in a dictionary

$P(w_{i}|Spam) = \frac{wi\_spam\_count\,+\,\alpha}{spam\_vocab\,+\,\alpha \cdot vocab}$

$\alpha$ is a coefficient that prevents a probability from being 0.


In [11]:
# Subtract 2 for "Email No." and "Prediction" columns
total_vocab = len(training_data.columns) - 2
total_vocab

2866

In [12]:
spam_training_emails = training_data.loc[training_data[PREDICTION] == 1]

total_spam_words = count_vocab(spam_training_emails)
total_spam_words

987979

In [13]:
ham_training_emails = training_data.loc[training_data[PREDICTION] == 0]

total_ham_words = count_vocab(ham_training_emails)
total_ham_words

1754023

In [14]:
word_spamicities = build_word_spamicity_dict(spam_training_emails, total_vocab, total_spam_words)
word_spamicities

{'ect': 0.0030923100989559416,
 'hou': 0.0007559204517356398,
 'enron': 1.009239588432096e-06,
 'com': 0.0035000428926825083,
 'gas': 0.00023111586575094996,
 'deal': 0.00020184791768641916,
 'meter': 3.532338559512336e-05,
 'hpl': 3.0277187652962875e-06,
 'please': 0.00042488986672991236,
 'e': 0.11977857283429799,
 'corp': 0.00018368160509464143,
 'know': 0.00020689411562857966,
 'need': 0.000315891991179246,
 'forwarded': 2.018479176864192e-06,
 'new': 0.0007983085144497879,
 'may': 0.0004117697520802951,
 'j': 0.004014755082782877,
 'mmbtu': 1.009239588432096e-06,
 'get': 0.0006438948574196771,
 'see': 0.00036736321018928286,
 'price': 0.0006126084301782822,
 'daren': 1.009239588432096e-06,
 'company': 0.000684264440956961,
 'l': 0.04481326544515035,
 'let': 0.0005873774404674798,
 'would': 0.00017560768838718467,
 'xls': 3.0277187652962875e-06,
 'farmer': 1.211087506118515e-05,
 'attached': 1.211087506118515e-05,
 'us': 0.003878507738344544,
 'information': 0.00044507465849855424,

In [15]:
word_hamicities = build_word_spamicity_dict(ham_training_emails, total_vocab, total_ham_words)
word_hamicities

{'ect': 0.00957374085670751,
 'hou': 0.003967809007854224,
 'enron': 0.0032056663796062243,
 'com': 0.0023120413412571882,
 'gas': 0.0013734504570294423,
 'deal': 0.0015834808004375916,
 'meter': 0.0011713887445365075,
 'hpl': 0.0016273082704712704,
 'please': 0.0011861876305219055,
 'e': 0.12650030821526004,
 'corp': 0.0008219073601121072,
 'know': 0.0006431823524423,
 'need': 0.0007245762253619893,
 'forwarded': 0.0005321907075518146,
 'new': 0.0004792562307578908,
 'may': 0.00048380973413801327,
 'j': 0.0064972801355122605,
 'mmbtu': 0.0006067543254013202,
 'get': 0.00048153298244795205,
 'see': 0.0005504047210723045,
 'price': 0.00045364277424470186,
 'daren': 0.0008811029040536995,
 'company': 0.00022995192069618514,
 'l': 0.045019918731348425,
 'let': 0.0007109157152216218,
 'would': 0.00037907915639519627,
 'xls': 0.0005116999423412634,
 'farmer': 0.0006033391978662283,
 'attached': 0.0005264988283266615,
 'us': 0.0028624460623294925,
 'information': 0.0002806096458000477,
 'mes

## Step 4: Calculate the "spamicity" and "hamicity" of each email

Multiply spamicities of each word together to get $\prod_{i=1}^{n}P(w_{i}|Spam)$.

Multiply that product by the probability that any email is spam.

In [16]:
test_data_spam_map = calculate_emails(testing_data, word_spamicities, spam_proportion)
test_data_spam_map
        

{0: -179.6143602253236,
 1: -6740.300340547724,
 2: -399.3454722655292,
 3: -3161.237436180195,
 4: -3182.851271107896,
 5: -3640.9160742095196,
 6: -1904.1304516323157,
 7: -1485.9147064294555,
 8: -1359.6334470866968,
 9: -3810.8486756760335,
 10: -5672.470273416164,
 11: -10719.80763702939,
 12: -6476.894790347868,
 13: -3280.428059470295,
 14: -1991.5817033972796,
 15: -2202.1067532000857,
 16: -535.4640290831345,
 17: -11293.800167329997,
 18: -1164.8085640905667,
 19: -2600.0148546154564,
 20: -818.4709949144514,
 21: -2247.5072703265987,
 22: -2878.490777597175,
 23: -878.4813799404327,
 24: -543.0076569122281,
 25: -12673.698138322485,
 26: -4100.643348273483,
 27: -1225.9418864702943,
 28: -5629.354413675017,
 29: -6315.001555639633,
 30: -1893.0840339754745,
 31: -470.4724632947332,
 32: -12576.848750051897,
 33: -3804.2041383490246,
 34: -2820.8438821723735,
 35: -1640.8626727088717,
 36: -3812.631242513441,
 37: -1361.9102915142241,
 38: -1575.9633370232157,
 39: -2902.1227

In [17]:
test_data_ham_map = calculate_emails(testing_data, word_hamicities, ham_proportion)
test_data_ham_map

{0: -176.15583964920575,
 1: -6432.041679849002,
 2: -357.41254536655924,
 3: -2932.900734827238,
 4: -2913.7099895922975,
 5: -3752.1900147086567,
 6: -1801.0952844645699,
 7: -1564.2419265205424,
 8: -1304.0300256287132,
 9: -3591.0561147109042,
 10: -5412.767535199986,
 11: -10351.139550102345,
 12: -6327.8408351694015,
 13: -3088.7980916380493,
 14: -1826.455990484612,
 15: -2134.57519266399,
 16: -533.7845757042195,
 17: -11515.183761933777,
 18: -1095.481086069865,
 19: -2394.443324781319,
 20: -769.362191311453,
 21: -2143.448309341941,
 22: -2573.918988487549,
 23: -840.2933458577579,
 24: -494.2465816941542,
 25: -13184.326904767382,
 26: -3920.9221319540857,
 27: -1154.7234883791152,
 28: -5454.7999748499615,
 29: -6033.217008538462,
 30: -1862.8210091251858,
 31: -488.7391997308602,
 32: -11877.26152334815,
 33: -3576.0902849544445,
 34: -2688.63272033158,
 35: -1554.905348643933,
 36: -3649.670464168598,
 37: -1256.336950142792,
 38: -1529.3518495493968,
 39: -2955.08157647

## Step 5: Compare hamicity and spamicity scores to classify emails

In [18]:
for key in test_data_spam_map:
    if test_data_spam_map[key] >= test_data_ham_map[key]:
        testing_data[CLASSIFICATION].loc[testing_data.index[key]] = 1
    else:
        testing_data[CLASSIFICATION].loc[testing_data.index[key]] = 0

## Step 6: Check accuracy of the model

In [19]:
score = calculate_accuracy(testing_data)
f'Accuracy: {score}%'

'Accuracy: 93.23017408123792%'